In [ ]:
import tensorflow as tf
import numpy as np
import statistical_analysis as sa
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn
import numpy as np
from sys import stdin
import csv


class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        self.phase_train = tf.placeholder(tf.bool, name='phase_train')

        # Keeping track of l2 regularization loss (optional)
        self.l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                conv = tf.nn.bias_add(conv, b)

                #batch normalization
                beta = tf.Variable(tf.constant(0.0, shape=[num_filters]),
                                   name='beta', trainable=True)
                gamma = tf.Variable(tf.constant(1.0, shape=[num_filters]),
                                    name='gamma', trainable=True)
                batch_mean, batch_var = tf.nn.moments(conv, [0, 1, 2], name='moments')
                ema = tf.train.ExponentialMovingAverage(decay=0.5)

                def mean_var_with_update():
                    ema_apply_op = ema.apply([batch_mean, batch_var])
                    with tf.control_dependencies([ema_apply_op]):
                        return tf.identity(batch_mean), tf.identity(batch_var)

                mean, var = tf.cond(self.phase_train,
                                    mean_var_with_update,
                                    lambda: (ema.average(batch_mean), ema.average(batch_var)))
                conv_bn = tf.nn.batch_normalization(conv, mean, var, beta, gamma, 1e-3)

                # Apply nonlinearity
                h = tf.nn.relu(conv_bn, name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs) #for tf_0.12
        #self.h_pool = tf.concat(pooled_outputs, 3) #for tf 1.2.1
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)



        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            self.l2_loss += tf.nn.l2_loss(W)
            self.l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.softmax(tf.nn.xw_plus_b(self.h_drop, W, b, name="scores"))
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y) #for tf_0.12
            #losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y) #for tf 1.2.1
            self.mean_loss = tf.reduce_mean(losses)
            self.loss = self.mean_loss + l2_reg_lambda * self.l2_loss


        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


#############################################play.py########################################################################################

# Parameters

# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("source", "tensorflow docker vscode bootstrap", "source file, split by space")
tf.flags.DEFINE_string("target", "DECA", "target file")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 192, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "2,3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.2, "L2 regularizaion lambda (default: 0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 50, "Number of training epochs (default: 1000)")
tf.flags.DEFINE_integer("evaluate_every", 1, "Evaluate model on dev set after this many epochs (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 5000, "Save model after this many steps (default: 100)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")


FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()


#sentence class
class_names = ( 'solution proposal', 'problem discovery')


def train_model(allow_save_model = False, print_intermediate_results = True):

    print("\nParameters:")
    for attr, value in sorted(FLAGS.__flags.items()):
        print("{}={}".format(attr.upper(), value))
    print("")

    # Data Preparatopn
    # ==================================================

    # Load data
    print("Loading data...")

    #load each source file
    dataset = FLAGS.source.split()
    source_text = np.array([])
    source_y = np.array([])
    for post in dataset:
        source_file_path = "data/" + post + "/"
        source_files = list()
        for class_name in class_names:
            source_files.append(source_file_path + class_name)
        tmp_text, tmp_y = data_helpers.load_data_and_labels(source_files)
        print post +": "+str(len(tmp_text))+" sentences"
        source_text = np.concatenate([source_text, tmp_text], 0)
        if len(source_y)==0:
            source_y = np.array(tmp_y)
        else:
            source_y = np.concatenate([source_y, tmp_y], 0)


    posts = FLAGS.target.split()
    target_text = np.array([])
    target_y = np.array([])
    for post in dataset:
        target_file_path = "data/" + post + "/"
        target_files = list()
        for class_name in class_names:
            target_files.append(target_file_path + class_name)
        tmp_text, tmp_y = data_helpers.load_data_and_labels(target_files)
        print post + ": " + str(len(tmp_text)) + " sentences"
        target_text = np.concatenate([target_text, tmp_text], 0)
        if len(target_y) == 0:
            target_y = np.array(tmp_y)
        else:
            target_y = np.concatenate([target_y, tmp_y], 0)


    all_text = np.concatenate([source_text, target_text], 0)

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in all_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    source_x = np.array(list(vocab_processor.fit_transform(source_text)))
    target_x = np.array(list(vocab_processor.fit_transform(target_text)))

    if print_intermediate_results:
        print('data distribution in source dataset')
        sa.print_data_distribution(source_y, class_names)
        print('data distribution in target dataset')
        sa.print_data_distribution(target_y, class_names)

        print("Max Document Length: {:d}".format(max_document_length))
        print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
        print("Train/Test size: {:d}/{:d}".format(len(source_y), len(target_y)))

    # Training
    # ==================================================

    min_loss = 100000000
    predictions_at_min_loss = None
    steps_per_epoch = (int)(len(source_y) / FLAGS.batch_size) + 1

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=FLAGS.allow_soft_placement,
            log_device_placement=FLAGS.log_device_placement)
        session_conf.gpu_options.allow_growth = True
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=source_x.shape[1],
                num_classes=source_y.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)

            learning_rate = tf.train.polynomial_decay(2*1e-3, global_step,
                                                      steps_per_epoch * FLAGS.num_epochs, 1e-4,
                                                      power=1)

            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            if allow_save_model:

                # Output directory for models and summaries
                timestamp = str(int(time.time()))
                out_dir = os.path.abspath(
                    os.path.join(os.path.curdir, "runs", FLAGS.source))
                print("Writing to {}\n".format(out_dir))

                # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
                checkpoint_dir_name = "checkpoint-" + str(FLAGS.embedding_dim) + "-" + FLAGS.filter_sizes + "-" + \
                                      str(FLAGS.num_filters) + "-" + str(FLAGS.dropout_keep_prob) + "-" + str(
                    FLAGS.l2_reg_lambda) + \
                                      "-" + str(FLAGS.batch_size) + "-" + str(FLAGS.num_epochs)
                checkpoint_dir = os.path.abspath(os.path.join(out_dir, checkpoint_dir_name))
                checkpoint_prefix = os.path.join(checkpoint_dir, "model")
                if not os.path.exists(checkpoint_dir):
                    os.makedirs(checkpoint_dir)
                saver = tf.train.Saver(tf.all_variables())

                # Write vocabulary
                vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer(), feed_dict={cnn.phase_train: True})  # this is for version r0.12

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                    cnn.input_x: x_batch,
                    cnn.input_y: y_batch,
                    cnn.dropout_keep_prob: FLAGS.dropout_keep_prob,
                    cnn.phase_train: True
                }
                _, step, loss, mean_loss, l2_loss, accuracy = sess.run(
                    [train_op, global_step, cnn.loss, cnn.mean_loss, cnn.l2_loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                # print("{}: step {}, loss {:g}, acc {:g}, mean_loss {}, l2_loss {}".format(time_str, step, loss, accuracy, mean_loss, l2_loss))
                return accuracy

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                    cnn.input_x: x_batch,
                    cnn.input_y: y_batch,
                    cnn.dropout_keep_prob: 1.0,
                    cnn.phase_train: False
                }
                step, loss, mean_loss, l2_loss, accuracy, predictions = sess.run(
                    [global_step, cnn.loss, cnn.mean_loss, cnn.l2_loss, cnn.accuracy, cnn.predictions],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()

                if print_intermediate_results:
                    print("{}: epoch {}, step {}, loss {:g}, acc {:g}, mean_loss {}, l2_loss {}".format(
                        time_str, step/steps_per_epoch, step, loss, accuracy, mean_loss, l2_loss))
                return accuracy, loss, predictions

            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(source_x, source_y)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...

            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_accuracy = train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                current_epoch = current_step/steps_per_epoch
                if current_step%steps_per_epoch==0 and current_epoch % FLAGS.evaluate_every == 0:
                    if print_intermediate_results:
                        print("Current train accuracy: %s\nEvaluation:" % (train_accuracy))

                    fold_accuracy, loss, predictions = dev_step(target_x, target_y)
                    #ensemble_prediction([predictions], target_y)
                    if loss < min_loss:
                        min_loss = loss
                        predictions_at_min_loss = predictions
                        if allow_save_model:
                            save_path = saver.save(sess, checkpoint_prefix)
                            if print_intermediate_results:
                                print("Model saved in file: %s" % save_path)


            # Final result
            output_file = open(target_file_path + 'fp_sentences', 'w')
            print('Final result:')
            fold_accuracy, loss, predictions = dev_step(target_x, target_y)
            print("ACC: %s" % (fold_accuracy))
            tp, fp, fn, precision, recall, f1 = sa.calculate_IR_metrics(target_text, target_y, predictions, class_names,
                                                                        output_file)
            for i in range(len(class_names)):
                print class_names[i], precision[i], recall[i], f1[i]
            #print("average f1-score: %s" % (sum(f1) / len(f1)))

            output_file.close()

    return min_loss, predictions_at_min_loss, target_y

def ensemble_prediction (list_predictions, y_classes):

    tp, fp, fn, precision, recall, f1 = (np.zeros(len(class_names)) for _ in range(6))

    num_instances = len(y_classes)
    votes = list()
    for _ in range(num_instances):
        votes.append((np.zeros(len(class_names))))


    for prediction_epoch in list_predictions:
        for instance_index, predicted_class in enumerate(prediction_epoch):
            votes[instance_index][predicted_class] += 1

    final_prediction = np.zeros(num_instances)

    for instance_index,true_class in enumerate(y_classes):
        true_class = np.argmax(true_class)
        predict_class = np.argmax(votes[instance_index])
        final_prediction[instance_index] = predict_class
        if predict_class == true_class:
            tp[true_class] += 1
        if predict_class != true_class:
            fn[true_class] += 1
            fp[predict_class] += 1

    #print ("final accuracy:",sum(tp)*1.0/len(y_classes))

    for i in range(len(class_names)):
        precision[i] = tp[i] / (tp[i] + fp[i])
        recall[i] = tp[i] / (tp[i] + fn[i])
        f1[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])

    for i in range(len(class_names)):
        print class_names[i], 'precision: ', precision[i], ', recall: ', recall[i], ', f1-score: ', f1[i]


def auto_param_tuning():


    tmp = FLAGS.embedding_dim
    best_embedding_dim = FLAGS.embedding_dim
    min_loss = 100000000
    list_embedding_dim = {128, 192, 256, 320}
    for embedding_dim in list_embedding_dim:
        FLAGS.embedding_dim = embedding_dim
        loss, predictions, target_y = train_model(False,False)
        if loss < min_loss:
            min_loss = loss
            best_embedding_dim = embedding_dim
    FLAGS.embedding_dim = tmp


    tmp = FLAGS.num_filters
    best_num_filters = FLAGS.num_filters
    min_loss = 100000000
    list_num_filters = {128, 192, 256, 320}
    for num_filters in list_num_filters:
        FLAGS.num_filters = num_filters
        loss, predictions, target_y = train_model(False,False)
        if loss < min_loss:
            min_loss = loss
            best_num_filters = num_filters
    FLAGS.num_filters = tmp


    FLAGS.embedding_dim = best_embedding_dim
    FLAGS.num_filters = best_num_filters

    list_predictions = list()
    list_filter_sizes = {'1,2,3','2,3,4','3,4,5','4,5,6','2,3,4,5','1,2,3,4','3,4,5,6','1,2,3,4,5','2,3,4,5,6','1,2,3,4,5,6'}
    for filter_sizes in list_filter_sizes:
        FLAGS.filter_sizes = filter_sizes
        loss, predictions, target_y = train_model(False,True)
        list_predictions.append(predictions)

    ensemble_prediction(list_predictions,target_y)


train_model(False,True)

#####################################################statistical_analysis.py################################################################################
def print_data_distribution (y_classes, class_names):
    """
    :param y_classes: class of each instance, for example, if there are 3 classes, and y[i] is [1,0,0], then instance[i] belongs to class[0]
    :param class_names: name of each class
    :return: None
    """
    count = np.zeros(len(class_names))
    for y in y_classes:
        class_index = np.argmax(y)
        count[class_index] = count[class_index] + 1
    for i, class_name in enumerate(class_names):
        print class_name,count[i]

def calculate_IR_metrics (x_text, y_classes, predictions, class_names, print_incorrect_predictions=None):
    """
    :param x_text: original text of each instance
    :param y_classes: true class of each instance, notice that y[i] is in form of array[n]
    :param predictions: index of predicted class of each instance
    :param class_names: name of each class
    :return: tp,fp,fn,precision,recall,f1
    """
    tp, fp, fn, precision, recall, f1 = (np.zeros(len(class_names)) for _ in range(6))
    for class_index in range(len(class_names)):
        for instance_index, predicted_class in enumerate(predictions):

            true_class = np.argmax(y_classes[instance_index]) #true class of this instance

            if true_class == class_index and predicted_class == class_index:
                tp[class_index] = tp[class_index] + 1

            if true_class == class_index and predicted_class != class_index:
                fn[class_index] = fn[class_index] + 1

            if true_class != class_index and predicted_class == class_index:
                fp[class_index] = fp[class_index] + 1

    for i in range(len(class_names)):
        precision[i] = tp[i] / (tp[i] + fp[i])
        recall[i] = tp[i] / (tp[i] + fn[i])
        f1[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i])

    if print_incorrect_predictions != None:

        print 'write incorrect preditions to: ',print_incorrect_predictions

        output_file = print_incorrect_predictions
        text_to_print = list()

        for instance_index, predicted_class in enumerate(predictions):
            true_class = np.argmax(y_classes[instance_index])
            if predicted_class != true_class:
                text_to_print.append(str(instance_index)+' True : '+class_names[true_class]+' Predicted: '+class_names[predicted_class]+' Text: '+x_text[instance_index])

        #text_to_print = sorted(text_to_print)
        for text in text_to_print:
            print >> output_file, text
            print (text)

        print >> output_file, '-----------------------------------------------------\n-----------------------------------------------------'


    return tp,fp,fn,precision,recall,f1
####################################################################################################################################

class_names = ( 'solution proposal', 'problem discovery')


# Checkpoint Parameters
tf.flags.DEFINE_string("checkpoint_dir", "runs/docker tensorflow bootstrap vscode/checkpoint-256-2,3,4,5-192-0.5-0.2-64-30", "Checkpoint directory from training run")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

# Map data into vocabulary
vocab_path = "runs/docker tensorflow bootstrap vscode/vocab"
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)



print FLAGS.checkpoint_dir
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
print ("current using model: " + FLAGS.checkpoint_dir)


#checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]
        phase_train = graph.get_operation_by_name("phase_train").outputs[0]
        conv = graph.get_operation_by_name("conv-maxpool-3").outputs


        # Tensors we want to evaluate
        predictions_tensor = graph.get_operation_by_name("output/predictions").outputs[0]

        print "input a sentence and the model will predict a label for you ^__^"
        while True:

            text = [stdin.readline()]
            x_test = np.array(list(vocab_processor.transform(text)))
            predictions = sess.run(predictions_tensor, {input_x: x_test, dropout_keep_prob: 1.0, phase_train: False})
            print predictions, class_names[predictions[0]]
            print ""
            print conv


In [ ]:
Final result:

problem discovery  precision=0.535 recall=0.510 f1=0.420
solution proposal  precision=0.472 recall=0.452 f1=0.448
